In [1]:
from google.oauth2 import service_account
from google.cloud import aiplatform

credentials = service_account.Credentials.from_service_account_file(
    'alura.json'
)

In [2]:
client = aiplatform.gapic.PredictionServiceClient(credentials=credentials)

In [3]:
import base64
from google.cloud.aiplatform.gapic.schema import predict

def predict_image_classification_sample(
    project: str,
    endpoint_id: str,
    filename: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    service_account_json: str = 'alura.json',
):
    # Crea los credenciales a partir del archivo con la clave del servicio
    credentials = service_account.Credentials.from_service_account_file(
        service_account_json
    )

    # Las opciones del cliente requiere puntos finales de API regionales.
    client_options = {"api_endpoint": api_endpoint}

    # Inicializa el cliente que será usado para crear y enviar solicitaciones
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options, credentials=credentials)

    with open(filename, "rb") as f:
        file_content = f.read()

    # El formato de cada instancia debe estar de acuerdo con el esquema de entrada de previsión del modelo implementado
    encoded_content = base64.b64encode(file_content).decode("utf-8")
    instance = predict.instance.ImageClassificationPredictionInstance(
        content=encoded_content,
    ).to_value()
    instances = [instance]

    # Consulte gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml para el formato de los parámetros.
    parameters = predict.params.ImageClassificationPredictionParams(
        confidence_threshold=0.5,
        max_predictions=5,
    ).to_value()
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )

    # Realiza la previsión
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )

    # Procesa la respuesta
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

# Ejemplo de como llamar la función:
# predict_image_classification_sample(
#     project="your-project-id",
#     endpoint_id="your-endpoint-id",
#     filename="/ruta/para/imagen.jpg",
#     service_account_json="/ruta/para/su-llave-servicio.json"
# )


In [4]:
predict_image_classification_sample(
    project="162384697599",
    endpoint_id="9167500353489338368",
    filename="Amanita_545.jpg",
    service_account_json="alura.json"
)

response
 deployed_model_id: 1067679666640257024
 prediction: {'confidences': [0.950009], 'displayNames': ['Amanita'], 'ids': ['3389008472497455104']}
